# Análise de Dados Exploratória da ANAC

![ANAC](imgs/anac-logo.png "Logo ANAC")

# 1. Introdução

Este estudo tem como objetivo analisar um conjunto de dados de voos internacionais registrados pela Agência Nacional de Aviação Civil (ANAC) do Brasil, durante o período de janeiro de 2015 a agosto de 2017. 

A ANAC é a agência reguladora responsável pela aviação civil no país, e as informações contidas nesses dados são de grande importância para a compreensão da dinâmica da aviação internacional no Brasil, bem como para a tomada de decisões estratégicas no setor. O conjunto de dados inclui informações como o número do voo, nome da companhia aérea, horários de partida e chegada previstos e reais, situação do voo e localização dos aeroportos de origem e destino, entre outros. Esses dados permitem uma análise detalhada das tendências e padrões de voos internacionais no Brasil, bem como a identificação de fatores que possam influenciar o desempenho da aviação internacional no país.

## 1.1. Objetivos

O objetivo deste estudo é realizar uma Análise Exploratória dos Dados (Exploratory Data Analysis - EDA) do conjunto de dados disposto em [bases/](bases/), e analisar dados de voos internacionais rastreados pela ANAC.

Com base nas informações disponíveis, pretendemos avaliar o desempenho das companhias aéreas em relação ao cumprimento dos horários previstos de partida e chegada, bem como investigar a frequência de cancelamentos e as justificativas apresentadas para esses casos. 

Também vamos examinar as rotas mais utilizadas e os destinos mais comuns para os voos internacionais saindo do Brasil, bem como avaliar a pontualidade e a eficiência dos aeroportos de origem e destino. 

1. [Análise 1](#Análise1)
2. [Análise 2](#Análise2)
3. [Análise 3](#Análise3)
4. [Análise 4](#Análise4)


# 2. Metodologia

Nesta seção será apresentado todo o processo de preparação, organização e limpeza de dados feito no *dataset* que possui os seguintes dados brutos. Ao final desta, apresentaremos os dados padronizados e devidamente tratados:

|        Coluna        | Tipo |                         Descrição                         |
|:--------------------:|:----:|:---------------------------------------------------------:|
|         Voos         |  str |                       Número do voo                       |
|    Companhia.Aerea   |  str |                  Nome da companhia aérea                  |
|   Codigo.Tipo.Linha  |  str |          Código do tipo de linha (internacional)          |
|   Partida.Prevista   |  str |                Horário previsto de partida                |
|     Partida.Real     |  str |                  Horário real de partida                  |
|   Chegada.Prevista   |  str |                Horário previsto de chegada                |
|     Chegada.Real     |  str |                  Horário real de chegada                  |
|     Situacao.Voo     |  str |        Situação do voo (realizado, cancelado, etc)        |
| Codigo.Justificativa |  str | Código de justificativa (caso o voo tenha sido cancelado) |
|   Aeroporto.Origem   |  str |               Código do aeroporto de origem               |
|     Cidade.Origem    |  str |                      Cidade de origem                     |
|     Estado.Origem    |  str |                      Estado de origem                     |
|      Pais.Origem     |  str |                       País de origem                      |
|   Aeroporto.Destino  |  str |               Código do aeroporto de destino              |
|    Cidade.Destino    |  str |                     Cidade de destino                     |
|    Estado.Destino    |  str |                     Estado de destino                     |
|     Pais.Destino     |  str |                      País de destino                      |
|       LongDest       |  str |             Longitude do aeroporto de destino             |
|        LatDest       |  str |              Latitude do aeroporto de destino             |
|       LongOrig       |  str |              Longitude do aeroporto de origem             |
|        LatOrig       |  str |              Latitude do aeroporto de origem              |


## 2.1. Configuração do Ambiente

### 2.1.1. Bibliotecas e Ferramentas Necessárias (Built-ins)

1. ```os```:       Para manipular os arquivos de base e seus respectivos diretórios de forma multiplataforma e eficiente.
2. ```re```:       Utilizado em expressões regulares durante o processo de limpeza dos dados.
3. ```enum```:     Auxilia nos processos de padronizar e regular algumas convenções do projeto.
4. ```datetime```: Auxilia nos processos de padronizar e regular algumas convenções do projeto

### 2.1.2. Bibliotecas e Ferramentas Necessárias (pip)

1. Pandas: Para manipular e exibir os dados de forma coesa e relata.   
    ```pip install pandas```       
    ```import pandas as pd```

2. JinJa2: Complemento de estilização do Pandas.   
    ```pip install Jinja2```

3. Matplotlib: Para exibição visual dos dados por meio de gráficos.   
    ```pip install matplotlib```   
    ```import matplotlib.pyplot as plt```

4. DescricaoColuna e DescricaoTipo: São classes estáticas compostar por Enums estabelecidos com base nas convenções de nome das colunas do *dataset*.   
   **Utils** possui complementos para tratativas de alguns campos.   
    ```from libs.utils import Utils```

### 2.1.3. Importações gerais para o projeto

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from libs.utils import Utils, Plot, Fligths

## 2.2. Carregamento da base

In [ ]:
caminho = os.path.join(
            os.path.dirname(
                os.path.abspath(
                    os.path.join(__name__, '.'))), 'bases')

arquivos = [arquivo for arquivo in os.listdir(caminho) if arquivo.endswith(".csv")]

dataframes = [pd.read_csv(os.path.join(caminho, arquivo)) for arquivo in arquivos]

flights_dataframe = pd.concat(dataframes)

## 2.3. Organização e Limpeza dos Dados

### 2.3.1. Padrozinação dos nomes das colunas

In [ ]:
nomes_colunas_tratadas = []

for coluna in flights_dataframe.columns:
    if any(char.isupper() for char in coluna):
        if coluna == 'Voos': coluna = 'Num.Voo'
        nomes_colunas_tratadas.append('_'.join(word.lower() for word in re.findall('[A-Z][^A-Z]*', coluna)).lower().replace('.', ''))

flights_dataframe.columns = nomes_colunas_tratadas

### 2.3.2. Padrozinação dos tipos das colunas de data/hora, string e númericos

In [ ]:
for coluna in flights_dataframe.columns:
    tipo = Utils.tipo_coluna(coluna)
    if tipo.__name__ == 'datetime': 
        flights_dataframe = Utils.formatar_colunas_datetime(flights_dataframe, coluna)

### 2.3.3. Padrozinação no nome das companhias aérea.

In [ ]:
flights_dataframe['companhia_aerea'] = flights_dataframe.apply(
    lambda row: Utils.nome_companhia_aerea_normalizado(
        row['companhia_aerea'], 
        ), axis=1)

### 2.3.4. Feature Engineering
A engenharia de recursos (*Feature Engineering*) é o processo de usar o conhecimento do domínio para extrair recursos dos dados brutos. Neste estudo serão acrescentadas as seguintes colunas para enriquecer a análise de dados:

| Coluna         | Descrição                                                            | Base         |
|:--------------:|:--------------------------------------------------------------------:|:--------------:|
| partida_atrasou | Houve atraso na partida (S/N)                                       | `partida_prevista - partida_real` |
| chegada_atrasou | Houve atraso na chegada (S/N)                                       | `chegada_prevista - chegada_real` |
| distancia_km   | Distância total em kms da origem ao destino                          | `lat_long_orig - lat_long_dest` |
| justificativa_atraso   | Informação e correlata à justificativa no atraso do voo      | `codigo_justificativa` |
| justificativa_cancelamento   | Informação e correlata à justificativa do cancelamento do voo| `codigo_justificativa` |


#### 2.3.3.1. Atraso na partida

In [ ]:
flights_dataframe['partida_atrasou'] = flights_dataframe.apply(
    lambda row: Utils.validar_atraso(
        row['situacao_voo'], 
        row['partida_prevista'], 
        row['partida_real']
        ), axis=1)

#### 2.3.3.2. Atraso na chegada

In [ ]:
flights_dataframe['chegada_atrasou'] = flights_dataframe.apply(
    lambda row: Utils.validar_atraso(
        row['situacao_voo'], 
        row['chegada_prevista'], 
        row['chegada_real']
        ), axis=1)

#### 2.3.3.3. distancia_km (para *lat_long_orig* e *lat_long_dest*)

In [ ]:
flights_dataframe['distancia_km'] = flights_dataframe.apply(
    lambda row: Utils.calcular_distancia_km(
    row['lat_orig'],
    row['long_orig'],
    row['lat_dest'],
    row['long_dest']
    ), axis=1)

#### 2.3.3.4. Justificativa atraso

In [ ]:
flights_dataframe['justificativa_atraso'] = ''
flights_dataframe['justificativa_atraso'] = np.where(
    ((flights_dataframe['partida_atrasou'] == 'S') | (flights_dataframe['chegada_atrasou'] == 'S')),
    flights_dataframe.apply(lambda row: Utils.motivo_atraso(row['codigo_justificativa']), axis=1),
    flights_dataframe['justificativa_atraso']
)

#### 2.3.3.5. Justificativa cancelamento

In [ ]:
flights_dataframe['justificativa_cancelamento'] = ''
flights_dataframe['justificativa_cancelamento'] = np.where(
    flights_dataframe['situacao_voo'] == 'Cancelado',
    flights_dataframe.apply(lambda row: Utils.motivo_cancelamento(row['codigo_justificativa']), axis=1),
    flights_dataframe['justificativa_cancelamento']
)

## 2.4. Mapeamento de Dados
Para melhorar a análise e interpretação dos dados os seguintes atributos serão modificados:

| Coluna    | Original     | Modificado                          |
|:----------:|:-------------:|:------------------------------------:|
| *    | 'NA', *      | '', *                  |
| *      | np.NaN / NaN      | ''                  |


### 2.4.1. Removendo 'N/I' por um valor ''/string.empty

In [ ]:
for coluna in flights_dataframe.columns:
    flights_dataframe[coluna] = flights_dataframe[coluna].replace('N/I', '')

### 2.4.2. Removendo np.NaN por um valor None

In [ ]:
flights_dataframe.fillna(value='', inplace=True)

### 2.5. Reordenando as colunas de forma semântica

In [ ]:
flights_dataframe = flights_dataframe.reindex(columns=Utils.nomes_colunas(), index=None)
flights_dataframe.drop('codigo_justificativa', axis=1, inplace=True)

dataframe = flights_dataframe

### 2.6 Resultado final

In [ ]:
columns = ['NomeColuna', 'DescricaoColuna', 'TipoColuna']
values = [[value, Utils.descricao_coluna(value), Utils.tipo_coluna(value).__name__] for value in dataframe.columns]

pd.DataFrame(data=values, columns=columns).style.set_table_styles([
            dict(selector='th', props=[('text-align', 'center')]), 
            dict(selector='td', props=[('text-align', 'center')])
            ])

# 3. Análise dos Dados

## 3.1. Quantidade de voos por companhia aérea.
Verificar a quantidade de voos realizados por cada companhia aérea.

In [ ]:
voos_por_cia_area = Fligths.voos_por_cia_area_groupby(dataframe, result_reset_index = True)

### 3.1.1. Quantidade de voos regionais por companhia aérea.

In [ ]:
%matplotlib inline
regionais = Fligths.voos_por_tipo_linha_groupby(dataframe=voos_por_cia_area, tipo_linha='Regional', result_reset_index=True)
regionais = regionais[['companhia_aerea', 'voos_realizados', 'voos_cancelados', 'voos_com_atraso']]
display(regionais.head(10))
Plot.barh_voos_por_cia_area_tipo_linha(regionais)

### 3.1.2. Quantidade de voos nacionais por companhia aérea.

In [ ]:
%matplotlib inline
nacionais = Fligths.voos_por_tipo_linha_groupby(dataframe=voos_por_cia_area, tipo_linha='Nacional', result_reset_index=True)
nacionais = nacionais[['companhia_aerea', 'voos_realizados', 'voos_cancelados', 'voos_com_atraso']]
display(nacionais.head(10))
Plot.barh_voos_por_cia_area_tipo_linha(nacionais)

### 3.1.3. Quantidade de voos internacionais por companhia aérea.

In [ ]:
%matplotlib inline
internacionais = Fligths.voos_por_tipo_linha_groupby(dataframe=voos_por_cia_area, tipo_linha='Internacional', result_reset_index=True)
internacionais = internacionais[['companhia_aerea', 'voos_realizados', 'voos_cancelados', 'voos_com_atraso']]
display(internacionais.style.hide_index())
Plot.barh_voos_por_cia_area_tipo_linha(internacionais)